Prompt Engineering with Amazon Bedrock

Based on the AWS Samples: Amazon Bedrock Module 1 - Text Generation, we want to introduce how prompt engineering are done in Amazon Bedrock.

Introduction

Welcome to this introduction to building conversational AI with Amazon Bedrock's Converse API! The primary goal of this chapter is to provide a comprehensive introduction to Amazon Bedrock APIs for generating text. While we'll explore various use cases like summarization and code generation, our focus is on understanding the API patterns.

In this notebook, you will:
* Learn the basics of the Amazon Bedrock Invoke API
* Explore the more powerful Converse API and it's features like multi-turn conversation, streaming, or  function calling
* Apply these APIs across various foundation models
* Compare results across different state-of-the-art models

Pre-requisites
This notebook requires permissions to access Amazon Bedrock.
Please ensure that you hav AWS-CLI SDK and IAM credentials to access Amazon Bedrock with your AWS account. 
Note: Please make sure to enable the following models in your Amazon Bedrock Console (Go to Model Access and select the models you want to use from Amazon Bedrock) before running this notebook:

Anthropic Claude 3.7 Sonnet
Anthropic Claude 3.5 Sonnet
Anthropic Claude 3.5 Haiku
Amazon Nova Pro
Amazon Nova Micro
DeepSeek-R1
Meta LLama 3.1 70B Instruct
This notebook uses these models to demonstrate various capabilities of the Amazon Bedrock APIs.